In [8]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import statistics as st
import seaborn as sns
import datetime 
import pandas as pd
import os
import sqlite3
from sqlite3 import Error

In [3]:
df = pd.read_csv("Sample-Superstore.csv", encoding = "latin1")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [5]:
df.nunique()

Row ID           9994
Order ID         5009
Order Date       1237
Ship Date        1334
Ship Mode           4
Customer ID       793
Customer Name     793
Segment             3
Country             1
City              531
State              49
Postal Code       631
Region              4
Product ID       1862
Category            3
Sub-Category       17
Product Name     1850
Sales            5825
Quantity           14
Discount           12
Profit           7287
dtype: int64

In [6]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [7]:
# Check for duplicates in OrderID and ProductID
duplicates = df[df.duplicated(subset=["Order ID", "Product ID"], keep=False)]

# Display duplicates
print(duplicates)


      Row ID        Order ID  Order Date   Ship Date       Ship Mode  \
350      351  CA-2016-129714    9/1/2016    9/3/2016     First Class   
352      353  CA-2016-129714    9/1/2016    9/3/2016     First Class   
430      431  US-2016-123750   4/15/2016   4/21/2016  Standard Class   
431      432  US-2016-123750   4/15/2016   4/21/2016  Standard Class   
1300    1301  CA-2016-137043  12/23/2016  12/25/2016    Second Class   
1301    1302  CA-2016-137043  12/23/2016  12/25/2016    Second Class   
3183    3184  CA-2017-152912   11/9/2017  11/12/2017    Second Class   
3184    3185  CA-2017-152912   11/9/2017  11/12/2017    Second Class   
3405    3406  US-2014-150119   4/23/2014   4/27/2014  Standard Class   
3406    3407  US-2014-150119   4/23/2014   4/27/2014  Standard Class   
6498    6499  CA-2015-103135   7/24/2015   7/28/2015  Standard Class   
6500    6501  CA-2015-103135   7/24/2015   7/28/2015  Standard Class   
7881    7882  CA-2017-118017   12/3/2017   12/6/2017    Second C

# Loading data into sqllite

In [14]:
# Connect to SQLite database (creates file if not exists)
conn = sqlite3.connect("superstore.db")
cursor = conn.cursor()


In [15]:
# Create Customers Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Customers (
    CustomerID TEXT PRIMARY KEY,
    CustomerName TEXT,
    Segment TEXT,
    Region TEXT,
    Country TEXT,
    City TEXT,
    State TEXT,
    PostalCode INTEGER
);
''')

# Create Regions Table 
cursor.execute('''
CREATE TABLE IF NOT EXISTS Regions (
    RegionID INTEGER PRIMARY KEY AUTOINCREMENT,
    RegionName TEXT
);
''')

# Create Categories Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Categories (
    CategoryID INTEGER PRIMARY KEY AUTOINCREMENT,
    CategoryName TEXT
);
''')

# Create Products Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Products (
    ProductID TEXT PRIMARY KEY,
    ProductName TEXT,
    CategoryID INTEGER,
    SubCategory TEXT,
    FOREIGN KEY (CategoryID) REFERENCES Categories(CategoryID)
);
''')

# Create Orders Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Orders (
    OrderID TEXT PRIMARY KEY,
    OrderDate DATE,
    CustomerID TEXT,
    ShipID INTEGER,
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);
''')

# Create Shipping Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Shipping (
    ShipID INTEGER PRIMARY KEY AUTOINCREMENT,
    ShipMode TEXT,
    ShipDate DATE
);
''')

# Create OrderDetails Table (Keeping OrderDetailID as PK)
cursor.execute('''
CREATE TABLE IF NOT EXISTS OrderDetails (
    OrderDetailID INTEGER PRIMARY KEY AUTOINCREMENT,
    OrderID TEXT,
    ProductID TEXT,
    Sales DECIMAL(10,2),
    Quantity INTEGER,
    Discount DECIMAL(5,2),
    Profit DECIMAL(10,2),
    FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),
    FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
);
''')

# Commit changes and close connection
conn.commit()
conn.close()


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Read the CSV file
df = pd.read_csv("Sample-Superstore.csv", encoding="latin1")

# Connect to SQLite
engine = create_engine("sqlite:///superstore.db")

### Insert Data into Regions ###
regions_df = df[['Region']].drop_duplicates().rename(columns={'Region': 'RegionName'})
regions_df.to_sql('Regions', con=engine, if_exists='append', index=False)

### Insert Data into Categories ###
categories_df = df[['Category']].drop_duplicates().rename(columns={'Category': 'CategoryName'})
categories_df.to_sql('Categories', con=engine, if_exists='append', index=False)

### Insert Data into Customers ###
customers_df = df[['Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region']].drop_duplicates()
regions_df = pd.read_sql("SELECT * FROM Regions", con=engine)  # Fetch Region IDs
customers_df = customers_df.merge(regions_df, left_on='Region', right_on='RegionName', how='left').drop(columns=['Region', 'RegionName'])
customers_df.to_sql('Customers', con=engine, if_exists='append', index=False)

### Insert Data into Products ###
products_df = df[['Product ID', 'Product Name', 'Category', 'Sub-Category']].drop_duplicates()
categories_df = pd.read_sql("SELECT * FROM Categories", con=engine)  # Fetch Category IDs
products_df = products_df.merge(categories_df, left_on='Category', right_on='CategoryName', how='left').drop(columns=['Category', 'CategoryName'])
products_df.to_sql('Products', con=engine, if_exists='append', index=False)

### Insert Data into Shipping ###
shipping_df = df[['Ship Mode', 'Ship Date']].drop_duplicates()
shipping_df.to_sql('Shipping', con=engine, if_exists='append', index=False)

### Insert Data into Orders ###
orders_df = df[['Order ID', 'Order Date', 'Customer ID', 'Ship Mode', 'Ship Date']].drop_duplicates()
shipping_df = pd.read_sql("SELECT * FROM Shipping", con=engine)  # Fetch Shipping IDs
orders_df = orders_df.merge(shipping_df, on=['Ship Mode', 'Ship Date'], how='left').drop(columns=['Ship Mode', 'Ship Date'])
orders_df.to_sql('Orders', con=engine, if_exists='append', index=False)

### Insert Data into OrderDetails ###
order_details_df = df[['Order ID', 'Product ID', 'Sales', 'Quantity', 'Discount', 'Profit']]
order_details_df.to_sql('OrderDetails', con=engine, if_exists='append', index=False)

print("✅ Database successfully created and loaded!")


In [12]:
import os

# Delete the old database file if it exists
if os.path.exists("superstore.db"):
    os.remove("superstore.db")
